In [ ]:
#Import all neccessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from astropy.io import fits
import astropy.units as u
from specutils import Spectrum1D
from specutils.manipulation import FluxConservingResampler
from astropy.nddata import NDData, StdDevUncertainty
from specutils.analysis import correlation
from astropy import modeling
from scipy.optimize import curve_fit
from datetime import date




#Create a function that retrieves all radial velocities from data gathered through Helium I line and H Alpha
def RadialVelocityCalculations():

    #List of names of datasets
    file_folders = ['2-23', '2-28', '3-12', '3-13', '3-21', '4-4']

    #Create two radial velocity lists, one for each line being examined
    global RV1
    global RV2
    global RV_Ave
    RV1 = []
    RV2 = []
    RV_Ave = []

    #Define a gaussian function that will be fit to each absorption to find the center of the line
    def Gauss(x, y0, a, x0, sigma):
        return y0 + a * np.exp(-(x - x0)**2 / (2 * sigma**2))

    #Define the bounds for the gaussian absorption fit for each line (Helium I and then Hydrogen Alpha)
    line_bounds = [[6630, 6700], [6550, 6600]]

    #Start a count and for loop to calculate each radial velocity for each line shift for both absoptions 
    f = 0
    for bounds in line_bounds:

        #Get the start and end bounds for the gaussian line fit
        start = bounds[0]
        end = bounds[1]

        #Define a list that will hold all spectra
        all_spectra = []

        #Start a counter to go through all files in a night's data folder
        i = 0
        #Start for loop for all folders of data
        for folder in file_folders:

            #Retrieve the names of all fits files into one list
            filelist = glob('AllData/' + folder + '/*.fit')
            filelist.sort(reverse = True)

            #Open the fits file and pull out the flux data
            full_spec = fits.open(filelist[10])
            flux = full_spec[0].data*u.Jy

            #Determine the length of data, wavelength start and wavelength steps
            length = flux.size
            wavelength_start = full_spec[0].header[7]
            wavelength_step = full_spec[0].header[6]
            waves = np.arange(wavelength_start, wavelength_start + length*wavelength_step, wavelength_step)*u.AA

            #Format into Spectrum1D 
            globals()[f'spec1d_{i}'] = Spectrum1D(spectral_axis=waves, flux=flux)

            #Append all spectra into thlist
            all_spectra.append(globals()[f'spec1d_{i}'])

            i = i + 1

        #Create the range for the absorption line
        new_disp_grid_0 = np.arange(start, end, 0.1) * u.AA
        
        #Use the flux conserving function to translate into the new bounds and create new Spectrum1D
        fluxcon = FluxConservingResampler()
        template_spec = fluxcon(spec1d_0, new_disp_grid_0) 
        template_spec = Spectrum1D(spectral_axis = np.log(template_spec.spectral_axis.to_value())*u.AA, flux = template_spec.flux, velocity_convention='optical')

        #Fit the gaussian
        mean = sum(template_spec.spectral_axis.to_value()*template_spec.flux.to_value())/sum(template_spec.flux.to_value())
        sigma = np.sqrt(sum(template_spec.flux.to_value() * (template_spec.spectral_axis.to_value() - mean)**2) / sum(template_spec.flux.to_value()))
        popt, pcov = curve_fit(Gauss, template_spec.spectral_axis.to_value(), template_spec.flux.to_value(), p0=[.7, max(template_spec.flux.to_value()), mean, sigma])

        #Determine the middle of the gaussian for the wavelength shift
        min_temp = template_spec.spectral_axis.to_value()[Gauss( template_spec.spectral_axis.to_value(), *popt).argmin()]

        #Start a counter and for loop to determine wavelength shift for all date of the same line
        j = 1
        while j < len(all_spectra):

            #Create the range for the absorption line
            new_disp_grid_1 = np.arange(start, end, 0.1) * u.AA

            #Use the flux converter function to translate into new bounds and create new Spectrum1D
            fluxcon = FluxConservingResampler()
            observe_spec = fluxcon(all_spectra[j], new_disp_grid_1)
            globals()[f'observe_spec{j}'] = Spectrum1D(spectral_axis = np.log(observe_spec.spectral_axis.to_value())*u.AA, flux = observe_spec.flux)
            
            #Fit the gaussian
            mean = sum(globals()[f'observe_spec{j}'].spectral_axis.to_value()*globals()[f'observe_spec{j}'].flux.to_value())/sum(globals()[f'observe_spec{j}'].flux.to_value())
            sigma = np.sqrt(sum(globals()[f'observe_spec{j}'].flux.to_value() * (globals()[f'observe_spec{j}'].spectral_axis.to_value() - mean)**2) / sum(globals()[f'observe_spec{j}'].flux.to_value()))
            popt, pcov = curve_fit(Gauss, globals()[f'observe_spec{j}'].spectral_axis.to_value(), globals()[f'observe_spec{j}'].flux.to_value(), p0=[.7, max(globals()[f'observe_spec{j}'].flux.to_value()), mean, sigma])

            #Determine the middle of the gaussian for the wavelength shift
            min_obs = globals()[f'observe_spec{j}'].spectral_axis.to_value()[Gauss(globals()[f'observe_spec{j}'].spectral_axis.to_value(), *popt).argmin()]

            j = j + 1

            #Calculate radial velocity from shift
            RV = abs(min_obs-min_temp)*(3*10**5)
            
            #Append radial velocity to appropriate list
            if f == 0:
                RV1.append(RV)
            else:
                RV2.append(RV)

        for rv1, rv2 in zip(RV1, RV2):
            RV_Ave.append((rv1+rv2)/2)
        
        
        f = f + 1

        
#Create a function to create times and phases to plot radial velocities
def DatesAndPhases():
    
    #Get each date for every observation
    time1 = date(2023, 2, 23)
    time2 = date(2023, 2, 28)
    time3 = date(2023, 3, 12)
    time4 = date(2023, 3, 13)
    time5 = date(2023, 3, 21)
    time6 = date(2023, 4, 4)

    #put all times into a list
    all_times = [time1, time2, time3, time4, time5, time6]
    
    #Define the known period of the binary
    period = 5.732436
    
    #Create two empty lists for days and shifts
    global phases
    global day_times
    day_times = []
    phases = []

    #Start a counter and while loop to make the day differences and phase fractions
    p = 1
    while p < 6:
        time_diff = all_times[p] - all_times[0]
        day_times.append(time_diff.days)
        phases.append(time_diff.days/period % 1)
        p += 1
    

#Create a function to plot the phases vs RV and days vs RV
def plotting():
    
    #Convert all lists to arrays
    phases_ = np.asarray(phases)
    day_times_ = np.asarray(day_times)
    RV1_ = np.asarray(RV1)
    RV2_ = np.asarray(RV2)
    RV_Ave_ = np.asarray(RV_Ave)
    
    #Make some initial guesses for fit
    guess_offset = 100
    guess_freq = 1
    guess_amplitude = 100
    guess_phase = 0
    p0=[guess_freq, guess_amplitude, guess_phase, guess_offset]

    #Define the cosine fit function
    def my_cos(x, freq, amplitude, phase, offset):
        return np.cos(x * freq + phase) * amplitude + offset
    
    #Fit both RVs with my cosine function and inital parameters
    fit1 = curve_fit(my_cos, phases_, RV1_, p0=p0)
    fit2 = curve_fit(my_cos, phases_, RV2_, p0=p0)
    fit3 = curve_fit(my_cos, phases_, RV_Ave_, p0=p0)
    data_fit1 = my_cos(np.arange(0, 1, 0.01), *fit1[0])
    data_fit2 = my_cos(np.arange(0, 1, 0.01), *fit2[0])
    data_fit3 = my_cos(np.arange(0, 1, 0.01), *fit3[0])

    #Plot phases with both RVs
    fig1, ax1 = plt.subplots(dpi=1600)
    
    ax1.xaxis.grid(True, which='minor')
    ax1.yaxis.grid(True, which='minor')
    ax1.scatter(phases_, RV1_-fit1[0][3], label='Helium I Data', color = 'blue')
    ax1.scatter(phases_, RV2_-fit2[0][3], label='Hydrogen Alpha Data', color = 'red')

    #Plot the functions
    #ax1.plot(np.arange(0, 1, 0.01), data_fit1-fit1[0][3], label='Helium I Fit', color = 'blue')
    #ax1.plot(np.arange(0, 1, 0.01), data_fit2-fit2[0][3], label='Hydrogen Alpha Fit', color = 'red')
    ax1.plot(np.arange(0, 1, 0.01), data_fit3-fit3[0][3], label='Average Radial Velocity Fit', color = 'green')
    
    #Make plot look nice
    
    
    ax1.set_ylim(-100, 100)
    ax1.set_xlim(-0.02, 1)
    
    #Make plot look nice
    ax1.set_xlabel('Orbital Phase', fontname='Times New Roman', fontsize=12)
    ax1.set_ylabel('Radial Velocity (km/s)', fontname='Times New Roman', fontsize=12)
    ax1.set_title('Radial Velocity Phase Diagram of Mintaka from 2-23-2023 to 4-4-2023', 
                  fontname='Times New Roman', fontsize=12)
    ax1.legend(prop={'family': 'Times New Roman'})
    
    plt.minorticks_on()
    plt.grid(visible=None, which='minor', axis='both')
    plt.tick_params(axis='x', direction='in')
    plt.tick_params(axis='y', direction='in')
    plt.tick_params(which='minor', axis='x', direction='in')
    plt.tick_params(which='minor', axis='y', direction='in')
    
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    plt.savefig('RV_OrbitalPhase_Curve.png')
    
    
    
    #Plot days with both RVs
    fig2, ax2 = plt.subplots(dpi=1600)
    ax2.xaxis.grid(True, which='minor')
    ax2.yaxis.grid(True, which='minor')
    fig2.autofmt_xdate()
    ax2.scatter(day_times_, RV1_-fit1[0][3], label='Helium I Data', color = 'blue')
    ax2.scatter(day_times_, RV2_-fit2[0][3], label='Hydrogen Alpha Data', color = 'red')

    fit3 = curve_fit(my_cos, day_times_, RV_Ave_, p0=p0)
    data_fit3 = my_cos(np.arange(0, 40, 0.01), *fit3[0])
    print(fit3[0])
    
    ax2.plot(np.arange(0, 40, 0.01), data_fit3-fit3[0][3], label='Average Radial Velocity Fit', color = 'green')
    
    #Make plot look nice
    ax2.set_ylim(-100, 100)
    ax2.set_xlim(-1, 42)
    
    #Make plot look nice
    ax2.set_xlabel('Day Number Since First Observation on 2-23-2023', fontname='Times New Roman', fontsize=12)
    ax2.set_ylabel('Radial Velocity (km/s)', fontname='Times New Roman', fontsize=12)
    ax2.set_title('Radial Velocity Curve of Mintaka from 2-23-2023 to 4-4-2023', fontname='Times New Roman', 
                 fontsize=12)
    ax2.legend(prop={'family': 'Times New Roman'})
    
    plt.minorticks_on()
    plt.grid(visible=None, which='minor', axis='both')
    plt.tick_params(axis='x', direction='in')
    plt.tick_params(axis='y', direction='in')
    plt.tick_params(which='minor', axis='x', direction='in')
    plt.tick_params(which='minor', axis='y', direction='in')
    plt.savefig('RV_DayNumber_Curve.png')
    


#Call al functions
RadialVelocityCalculations()
DatesAndPhases()
plotting()